In [5]:
#调用电脑的摄像头
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
while(True):
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



In [4]:
#将摄像头录取的视频保存

import numpy as np
import cv2

cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(r'D:\python数据\close.mp4',fourcc,20.0,(640,480))   #保存位置
while(cap.isOpened()):
    ret,frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,0)
        out.write(frame)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

In [2]:
#将视频以图片形式存储

import cv2
import os

def video2imgs(videoPath, imgPath):
    if not os.path.exists(imgPath):
        os.makedirs(imgPath)             # 目标文件夹不存在，则创建
    cap = cv2.VideoCapture(videoPath)    # 获取视频
    judge = cap.isOpened()                 # 判断是否能打开成功
    print(judge)
    fps = cap.get(cv2.CAP_PROP_FPS)      # 帧率，视频每秒展示多少张图片
    print('fps:',fps)

    frames = 1                           # 用于统计所有帧数
    count = 1                            # 用于统计保存的图片数量

    while(judge):
        flag, frame = cap.read()         # 读取每一张图片 flag表示是否读取成功，frame是图片
        if not flag:
            print(flag)
            print("Process finished!")
            break
        else:
            if frames % 10 == 0:         # 每隔10帧抽一张
                imgname = 'jpgs_' + str(count).rjust(3,'0') + ".jpg"
                newPath = imgPath + imgname
                print(imgname)
                cv2.imwrite(newPath, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
                # cv2.imencode('.jpg', frame)[1].tofile(newPath)a
                count += 1
        frames += 1
    cap.release()
    print("共有 %d 张图片"%(count-1))
video2imgs(r'D:\python数据\QQ录屏20210108213638.mp4','./jpgs/')

True
fps: 20.0
jpgs_001.jpg
jpgs_002.jpg
jpgs_003.jpg
jpgs_004.jpg
jpgs_005.jpg
jpgs_006.jpg
jpgs_007.jpg
jpgs_008.jpg
jpgs_009.jpg
jpgs_010.jpg
jpgs_011.jpg
jpgs_012.jpg
jpgs_013.jpg
jpgs_014.jpg
jpgs_015.jpg
jpgs_016.jpg
False
Process finished!
共有 16 张图片


In [1]:
#将视频以图片形式存储
import tensorflow as tf
from PIL import Image
import cv2
import os
from keras.models import load_model
from keras import models
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from keras import layers
from keras import models 
from keras import optimizers

class Inception(tf.keras.layers.Layer):
    def __init__(self,c1,c2,c3,c4):
        super().__init__()
        self.p1_1=tf.keras.layers.Conv2D(c1,kernel_size=3,activation='relu',padding='same')
        self.p2_1=tf.keras.layers.Conv2D(c2[0],kernel_size=3,padding='same',activation='relu')
        self.p2_2=tf.keras.layers.Conv2D(c2[1],kernel_size=(3,3),padding='same',activation='relu')
        self.p3_1=tf.keras.layers.Conv2D(c3[0],kernel_size=1,padding='same',activation='relu')
        self.p3_2=tf.keras.layers.Conv2D(c3[1],kernel_size=(3,3),padding='same',activation='relu')
        self.p4_2=tf.keras.layers.Conv2D(c4,kernel_size=1,padding='same',activation='relu')
        
    def call(self,x):
        p1=self.p1_1(x)
        p2=self.p2_2(self.p2_1(x))
        p3=self.p3_2(self.p3_1(x))
        p4=self.p4_2(x)
        return tf.concat([p1,p2,p3,p4],axis=-1)

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Conv2D(128,(3,3),activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))
model.add(Inception(128,(128,256),(256,512),512))
model.add(layers.Flatten())
# model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
# model.summary()
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])

model.load_weights(r'C:\Users\24132\python深度学习\open_and_close_samll_1.h5')


# model = load_model(r'C:\Users\24132\python深度学习\open_and_close_samll_1.h5')
def video2imgs(videoPath, imgPath):
    result = []
    if not os.path.exists(imgPath):
        os.makedirs(imgPath)             # 目标文件夹不存在，则创建
    cap = cv2.VideoCapture(videoPath)    # 获取视频
    judge = cap.isOpened()                 # 判断是否能打开成功
    print(judge)
    fps = cap.get(cv2.CAP_PROP_FPS)      # 帧率，视频每秒展示多少张图片
    print('fps:',fps)

    frames = 1                           # 用于统计所有帧数
    count = 1                            # 用于统计保存的图片数量
    count_1 = 0
    while(judge):
        flag, frame = cap.read()         # 读取每一张图片 flag表示是否读取成功，frame是图片
        if not flag:
            print(flag)
            print("Process finished!")
            break
        else:
            if frames % 10 == 0:         # 每隔10帧抽一张
                imgname = 'jpgs_' + str(count).rjust(3,'0') + ".jpg"
                newPath = imgPath + imgname
                print(imgname)
                cv2.imwrite(newPath, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
                # cv2.imencode('.jpg', frame)[1].tofile(newPath)a
                pri_image = Image.open(newPath)
                pri_image.rotate(180).save(newPath)
                img = image.load_img(newPath,target_size=(150,150))
                img_tensor = image.img_to_array(img)
                img_tensor = np.expand_dims(img_tensor,axis=0)
                img_tensor /= 255.
                print(img_tensor.shape)
                result.append(model.predict_classes(img_tensor))
                if model.predict_classes(img_tensor)==[1]:
                    count_1 += 1
                count += 1
        frames += 1
    cap.release()
    print("共有 %d 张图片"%(count-1))
    print('测试睁眼的图片有：%d张'%count_1)
    print(result)
# video2imgs(r'D:\qq\2690486436\FileRecv\data\2.mp4','./jpgs_2/')

# file = r'C:\Users\24132\python模块学习\opencv\jpgs_1\jpgs_001.jpg'
# img = image.load_img(file,target_size=(150,150))
# img_tensor = image.img_to_array(img)
# img_tensor = np.expand_dims(img_tensor,axis=0)
# img_tensor /= 255.
# print(model.predict_classes(img_tensor))

base_file = 'C:/Users/24132/python模块学习/opencv/jpgs_1/'
result = 0
sum = 0
for i in os.listdir(base_file):
    img = image.load_img(base_file+i,target_size=(150,150))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor,axis=0)
    img_tensor /= 255.
    sum += 1
    if model.predict_classes(img_tensor)==[1]:
        result += 1
print('共有图片{0}张，其中睁眼图片{1}张，闭眼图片{2}张'.format(sum,result,sum-result))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
共有图片179张，其中睁眼图片179张，闭眼图片0张


In [ ]:
from keras.preprocessing import image
img = image.load_img(img_path,target_size=(150,150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor,axis=0)
img_tensor /= 255.